In [1]:
# Imports

# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook


In [2]:
with open('/Users/javwyk/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()


dict_keys(['api_key', 'api_read_token'])

In [3]:
tmdb.API_KEY =  login['api_key']


In [4]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # Save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    for c in releases['countries']:
        # If the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            ## Save a certification" key in info with the certification
            info['certification'] = c['certification']
            
    return info

In [5]:
FOLDER = "C:/Users/javwyk/OneDrive - MMI HOLDINGS LTD/GitHub/Github-practice-assignment/data/final"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)


['final_tmdb_data_2016.csv.gz',
 'final_tmdb_data_2017.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2016.json',
 'tmdb_api_results_2017.json',
 'tmdb_api_results_2018.json',
 'tmdb_api_results_2019.json',
 'tmdb_api_results_2021.json',
 'tmdb_api_results_2022.json']

In [6]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [7]:
# Load in the dataframe from project part 1 as basics:
# basics = pd.read_csv("Data/title_basics.csv.gz")
basics = pd.read_csv(f"{FOLDER}/title_basics.csv.gz")


In [8]:
YEARS_TO_GET = [2020]


In [9]:
errors = [ ]


In [10]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):

    #Defining the JSON file to store results for year
    # JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    JSON_FILE = f'{FOLDER}/tmdb_api_results_{YEAR}.json'
    print(JSON_FILE)

    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    print(file_exists)
    
    
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    print(movie_ids)

    display(df.head())
    

    # Load existing data from json into a dataframe called "previous_df"
    # previous_df = pd.read_json(JSON_FILE)
    previous_df = pd.read_json(JSON_FILE)

    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

    # final_year_df = pd.read_json(JSON_FILE)
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}/final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/1 [00:00<?, ?it/s]

C:/Users/javwyk/OneDrive - MMI HOLDINGS LTD/GitHub/Github-practice-assignment/data/final/tmdb_api_results_2020.json
False
1        tt0062336
11872    tt0805647
13411    tt0920462
13459    tt0926132
13985    tt0983946
           ...    
87342    tt9904802
87352    tt9911196
87356    tt9914192
87361    tt9916190
87362    tt9916362
Name: tconst, Length: 5031, dtype: object


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
11872,tt0805647,movie,The Witches,The Witches,0,2020,NaN,106,"Adventure,Comedy,Family"
13411,tt0920462,movie,Lost Girls and Love Hotels,Lost Girls and Love Hotels,0,2020,NaN,97,Drama
13459,tt0926132,movie,Road to Red,Road to Red,0,2020,NaN,120,"Action,Adventure,Horror"
13985,tt0983946,movie,Fantasy Island,Fantasy Island,0,2020,NaN,109,"Fantasy,Horror,Mystery"


Movies from 2020:   0%|          | 0/5031 [00:00<?, ?it/s]

ValueError: Expected object or value